In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import numpy as np
import seaborn as sns
import matplotlib.ticker as ticker

In [2]:
#Import CSV and Create Ticker List
SP500 = pd.read_csv('Resources/companylistsp500.csv')
#nasdaq = pd.read_csv('Resources/NASDAQ.txt')
tickers = list(SP500['Symbol'])


In [7]:
#Import CSV and Create Ticker List
fundsfile = pd.read_csv('Resources/funds.csv')
#nasdaq = pd.read_csv('Resources/NASDAQ.txt')
funds = list(fundsfile['Symbol'])


In [8]:
def sp500_returns(symbols,period = '5y'):
    tickers_profit = []
    for x in symbols:
        ticker = yf.Ticker(x)
        hist = ticker.history(period=period).dropna()
        profit = (hist.loc[hist.index[len(hist)-1], 'Close']/hist.loc[hist.index[0], 'Close'])-1
        tickers_profit.append(profit)
    sp5y = pd.DataFrame()
    sp5y['Ticker'] = list(symbols)
    sp5y['5yr Return'] = tickers_profit
    

    return sp5y




In [9]:

fundsDF = sp500_returns(symbols = funds,period = '20y')

AttributeError: 'float' object has no attribute 'upper'

In [ ]:
#generate random portfolios with nmonkeys number of portfolios and nstocks number of stocks

def dart_monkeys(symbols,nmonkeys = 10,nstocks = 30):
    #set random weights for each stock in monkey portfolio
    weights = []
    for i in range(nmonkeys):
        randomlist = rd.sample(range(1, 99), nstocks)
        dby = sum(randomlist)
        rws = []
        for x in randomlist:
            rw = x/dby
            rws.append(rw)
        weights.append(rws)
    #Select random stocks for each stock in monkey portfolio
    stocks = []
    for stock in range(nmonkeys):
        randomS = rd.sample(symbols, nstocks)
        stocks.append(randomS)
    out = []
    #Generate DICT list with random stocks and weights
    for i in range (nmonkeys):
        d = dict(zip(stocks[i],weights[i]))
        out.append(d)
    
    return out


In [ ]:
changos = dart_monkeys(nmonkeys = 100000,nstocks = 10,symbols = tickers)

In [ ]:
#definir en documentación estructura de DF
def monkey_returns(DF,diclist):
    returns = []
    for x in diclist:
        #Obtaining Weights
        suma = 0
        for y in x.keys():
            #Sum 
            suma += float(DF[DF['Ticker'] == y]['5yr Return']) * x[y]
        suma = suma/len(x.keys())
        returns.append(suma)

    return returns


In [ ]:
changosr = monkey_returns(tickersDF,changos)

In [ ]:
def annualizer(listOfReturns,YearlyPeriod):
    annualizedr = []
    for x in listOfReturns:    
        a = x/YearlyPeriod
        annualizedr.append(a)
    return annualizedr

In [ ]:
sns.boxplot(data = annualizedr)

In [ ]:
seriesr = pd.Series(annualizedr) 
seriesr.describe()

In [ ]:
# fixed bin size
bins = np.arange(-1, 1, 0.001) # fixed bin size

plt.xlim([-0.05,0.75])

plt.hist(annualizedr, bins=bins, alpha=1)
plt.title('Random Stocks Return')
plt.xlabel("Annualized Return")
plt.ylabel('Count')
plt.axvline(seriesr.mean(), color='k', linestyle='dashed', linewidth=2, animated = True)
plt.axvline(seriesr.median(), color='g', linestyle='dashed', linewidth=2, animated = True)
plt.axvline(0.032, color='y', linestyle='dashed', linewidth=2, animated = True)
plt.axvline(0.064, color='r', linestyle='dashed', linewidth=2, animated = True)
plt.show()

In [ ]:
sns.distplot(annualizedr, hist=False, rug=False)

In [ ]:
def no_ticker_data(symbols,period = '5y'):
    tickers_profit = []
    Not_Available_Data = []
    for x in symbols:
        try:
            ticker = yf.Ticker(x)
            hist = ticker.history(period=period).dropna()
            profit = (hist.loc[hist.index[len(hist)-1], 'Close']/hist.loc[hist.index[0], 'Close'])-1
            tickers_profit.append(profit)
        
        except:
            Not_Available_Data.append(x)
    return Not_Available_Data


In [ ]:
NAs = no_ticker_data(symbols = funds,period = '2d')

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
yroutliers = remove_outlier(andf,0)
sns.boxplot(data = yroutliers)
plt.axhline(0.05892, color='k', linestyle='dashed', linewidth=2, animated = True,label='Mean')
#plt.axhline(0.0509, color='violet', linestyle='dashed', linewidth=2, animated = True,label='Median')
plt.axhline(0.032, color='y', linestyle='dashed', linewidth=2, animated = True, label='USD Inflation')
plt.axhline(0.064, color='r', linestyle='dashed', linewidth=2, animated = True,label='SPY Return')
plt.legend(loc ='upper right')
plt.title(label = 'Monkey Returns')


In [ ]:
yroutliers.median()